In [33]:
import itertools
import numpy as np
import pandas as pd 
from numbers import Number
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')

import pickle
import os
from statsmodels.formula.api import ols

In [2]:
im_db = 'data/im.db'

im_db_conn = sqlite3.connect(im_db)

In [3]:
q = """

SELECT *
FROM sqlite_master


;
"""

pd.read_sql(q, im_db_conn)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [4]:
budgets_df = pd.read_csv('data/tn.movie_budgets.csv')

In [5]:
budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


The below is filtering our data to only include movies with 10000 votes on IMDB, representing a healthy audience level. $$$$$$$

In [6]:
q = """
SELECT numvotes, primary_title
FROM movie_ratings
    LEFT JOIN movie_basics
        USING(movie_id)
WHERE numvotes > 9999
;
"""

pd.read_sql(q, im_db_conn)

,numvotes,primary_title
0,50352,The Legend of Hercules
1,326657,Moneyball
2,87288,Hereafter
3,428142,Mission: Impossible - Ghost Protocol
4,477771,21 Jump Street
...,...,...
2673,32800,Crimea
2674,18768,Boy Erased
2675,11168,The Insult
2676,22399,The Silence


In [7]:
q = """

SELECT typeof(person_id)
FROM writers

;
"""

pd.read_sql(q, im_db_conn)

,typeof(person_id)
0,text
1,text
2,text
3,text
4,text
...,...
255868,text
255869,text
255870,text
255871,text


Top Writers table

In [8]:
q = """
SELECT persons.person_id, persons.primary_name, AVG(movie_ratings.averagerating) AS writer_rating
FROM persons
LEFT JOIN writers ON persons.person_id = writers.person_id
LEFT JOIN movie_basics ON writers.movie_id = movie_basics.movie_id
LEFT JOIN movie_ratings ON movie_basics.movie_id = movie_ratings.movie_id
WHERE numvotes > 9999 and averagerating != 'NaN'
GROUP BY persons.person_id, persons.primary_name
HAVING writer_rating > 7 and COUNT(writers.movie_id) > 5
ORDER BY writer_rating DESC
"""

writers_df = pd.read_sql(q, im_db_conn)
writers_df.to_csv('data/writers.csv', index=False)

Top Directors table

In [9]:
q = """
SELECT persons.person_id, persons.primary_name, AVG(movie_ratings.averagerating) AS director_rating, numvotes
FROM persons
LEFT JOIN directors ON persons.person_id = directors.person_id
LEFT JOIN movie_basics ON directors.movie_id = movie_basics.movie_id
LEFT JOIN movie_ratings ON movie_basics.movie_id = movie_ratings.movie_id
WHERE numvotes > 9999 and averagerating != 'NaN'
GROUP BY persons.person_id, persons.primary_name
HAVING director_rating > 7 and COUNT(directors.movie_id) > 5
ORDER BY director_rating DESC
"""

directors_df = pd.read_sql(q, im_db_conn)
directors_df.to_csv('data/directors.csv', index=False)

In [10]:
q = """

SELECT primary_title as movie, genres, averagerating
FROM movie_basics
LEFT JOIN movie_ratings
    USING(movie_id)
;
"""

genres_df = pd.read_sql(q, im_db_conn)
print(genres_df)

                                              movie                genres  \
0                                         Sunghursh    Action,Crime,Drama   
1                   One Day Before the Rainy Season       Biography,Drama   
2                        The Other Side of the Wind                 Drama   
3                                   Sabse Bada Sukh          Comedy,Drama   
4                          The Wandering Soap Opera  Comedy,Drama,Fantasy   
...                                             ...                   ...   
146139                          Kuambil Lagi Hatiku                 Drama   
146140  Rodolpho Teóphilo - O Legado de um Pioneiro           Documentary   
146141                              Dankyavar Danka                Comedy   
146142                                       6 Gunn                  None   
146143               Chico Albuquerque - Revelações           Documentary   

        averagerating  
0                 7.0  
1                 7.2  
2  

In [11]:
list(genres_df['genres'][0].split(','))


['Action', 'Crime', 'Drama']

In [12]:
genres_df.dropna(subset=['genres'], inplace=True)
genres_df.dropna(subset=['averagerating'], inplace=True)

In [13]:
print(genres_df)

                                  movie                genres  averagerating
0                             Sunghursh    Action,Crime,Drama            7.0
1       One Day Before the Rainy Season       Biography,Drama            7.2
2            The Other Side of the Wind                 Drama            6.9
3                       Sabse Bada Sukh          Comedy,Drama            6.1
4              The Wandering Soap Opera  Comedy,Drama,Fantasy            6.5
...                                 ...                   ...            ...
146114                     Swarm Season           Documentary            6.2
146115                 Diabolik sono io           Documentary            6.2
146122                Sokagin Çocuklari          Drama,Family            8.7
146125                        Albatross           Documentary            8.5
146134                       Drømmeland           Documentary            6.5

[73052 rows x 3 columns]


In [14]:
genres_df['genre_list'] = genres_df['genres'].map(lambda x: list(x.split(',')))

In [15]:
print(genres_df['genre_list'])

0           [Action, Crime, Drama]
1               [Biography, Drama]
2                          [Drama]
3                  [Comedy, Drama]
4         [Comedy, Drama, Fantasy]
                    ...           
146114               [Documentary]
146115               [Documentary]
146122             [Drama, Family]
146125               [Documentary]
146134               [Documentary]
Name: genre_list, Length: 73052, dtype: object


In [16]:
genres_df = genres_df.explode('genre_list')

In [17]:
genres_df['genre_list'].value_counts()

Drama          30788
Documentary    17753
Comedy         17290
Thriller        8217
Horror          7674
Action          6988
Romance         6589
Crime           4611
Adventure       3817
Biography       3809
Family          3412
Mystery         3039
History         2825
Sci-Fi          2206
Fantasy         2126
Music           1968
Animation       1743
Sport           1179
War              853
Musical          721
News             579
Western          280
Reality-TV        17
Adult              3
Game-Show          2
Short              1
Name: genre_list, dtype: int64

In [18]:
genres_df.head()

,movie,genres,averagerating,genre_list
0,Sunghursh,"Action,Crime,Drama",7.0,Action
0,Sunghursh,"Action,Crime,Drama",7.0,Crime
0,Sunghursh,"Action,Crime,Drama",7.0,Drama
1,One Day Before the Rainy Season,"Biography,Drama",7.2,Biography
1,One Day Before the Rainy Season,"Biography,Drama",7.2,Drama


In [19]:
budgets_df = pd.read_csv("data/tn.movie_budgets.csv")
budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [20]:
movies_df = pd.read_csv("data/tmdb.movies.csv")
movies_df.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [21]:
movies_df.rename(columns = {'original_title':'movie'},inplace = True)

In [22]:
movies_and_budgets_df = movies_df.merge(budgets_df, how='inner', on = 'movie')
movies_and_budgets_df.head()

,Unnamed: 0,genre_ids,id_x,original_language,movie,popularity,release_date_x,title,vote_average,vote_count,id_y,release_date_y,production_budget,domestic_gross,worldwide_gross
0,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,30,"Mar 26, 2010","$165,000,000","$217,581,232","$494,870,992"
1,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,15,"May 7, 2010","$170,000,000","$312,433,331","$621,156,389"
2,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,37,"Nov 22, 1995","$30,000,000","$191,796,233","$364,545,516"
3,2473,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,37,"Nov 22, 1995","$30,000,000","$191,796,233","$364,545,516"
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,38,"Jul 16, 2010","$160,000,000","$292,576,195","$835,524,642"


In [26]:
movies_budgets_genres_df = movies_and_budgets_df.merge(genres_df, how='inner', on = 'movie')
movies_budgets_genres_df.head()

,Unnamed: 0,genre_ids,id_x,original_language,movie,popularity,release_date_x,title,vote_average,vote_count,id_y,release_date_y,production_budget,domestic_gross,worldwide_gross,genres,averagerating,genre_list
0,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,30,"Mar 26, 2010","$165,000,000","$217,581,232","$494,870,992","Action,Adventure,Animation",8.1,Action
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,30,"Mar 26, 2010","$165,000,000","$217,581,232","$494,870,992","Action,Adventure,Animation",8.1,Adventure
2,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,30,"Mar 26, 2010","$165,000,000","$217,581,232","$494,870,992","Action,Adventure,Animation",8.1,Animation
3,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,15,"May 7, 2010","$170,000,000","$312,433,331","$621,156,389","Action,Adventure,Sci-Fi",7.0,Action
4,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,15,"May 7, 2010","$170,000,000","$312,433,331","$621,156,389","Action,Adventure,Sci-Fi",7.0,Adventure


In [38]:
movies_budgets_genres_df.to_csv('data/movies_budgets_genres.csv')

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [24]:
q = """
SELECT genres, AVG(movie_ratings.averagerating) AS genre_rating
FROM movie_basics
LEFT JOIN movie_ratings ON movie_basics.movie_id = movie_ratings.movie_id
WHERE numvotes > 9999 and averagerating != 'NaN'
GROUP BY movie_basics.genres
HAVING COUNT(genres) > 20
ORDER BY genre_rating DESC
"""

df = pd.read_sql(q, im_db_conn)
print(df)

                        genres  genre_rating
0                  Documentary      7.400000
1   Action,Adventure,Animation      7.356000
2       Biography,Comedy,Drama      7.218519
3                        Drama      7.208163
4      Biography,Drama,History      7.133333
5                  Crime,Drama      7.121429
6                Drama,Romance      6.909091
7        Biography,Crime,Drama      6.903846
8       Drama,Mystery,Thriller      6.885714
9              Biography,Drama      6.880645
10                Comedy,Drama      6.859350
11     Action,Adventure,Sci-Fi      6.713333
12        Crime,Drama,Thriller      6.710638
13          Comedy,Crime,Drama      6.708333
14         Crime,Drama,Mystery      6.675676
15              Drama,Thriller      6.630000
16  Adventure,Animation,Comedy      6.579167
17        Comedy,Drama,Romance      6.546825
18       Action,Crime,Thriller      6.518750
19          Action,Crime,Drama      6.513333
20       Action,Drama,Thriller      6.500000
21     Act

In [25]:
df.explode()

TypeError: explode() missing 1 required positional argument: 'column'

In [ ]:
q = """

SELECT *, AVG(averagerating) As avg_rating
FROM movie_basics
LEFT JOIN movie_ratings
    USING (movie_id)
    LEFT JOIN writers
        USING(movie_id)
            LEFT JOIN persons
                USING(person_id)
GROUP BY writers.person_id
HAVING avg_rating != 'NaN' and
ORDER BY avg_rating
;
"""

pd.read_sql(q, im_db_conn)

In [ ]:
q = """

SELECT *
FROM persons
LEFT JOIN directors
    USING (person_id)
    LEFT JOIN writers
        USING(person_id)
WHERE primary_profession LIKE '%director%' or primary_profession LIKE '%writer%'
GROUP BY person_id
;
"""

pd.read_sql(q, im_db_conn)

In [ ]:
df = pd.read_sql(q, im_db_conn)

In [ ]:
im_db_conn.close()

In [ ]:
sns.scatterplot(data=df, x = 'genres', y='averagerating')
plt.xlabel('averagerating')
plt.ylabel('numvotes')
plt.show()

In [ ]:
plt.plot(df['genres'], df['averagerating'])
plt.xlabel('averagerating')
plt.ylabel('numvotes')
plt.title('Plot of Data from SQL Database')
plt.show()

In [ ]:
df.info()

In [ ]:
q = """

SELECT *
FROM persons
WHERE primary_profession LIKE '%director%' OR primary_profession LIKE '%writer%'
GROUP BY Primary_name
;
"""

pd.read_sql(q, im_db_conn)

In [ ]:
q = """

SELECT *
FROM movie_basics

;
"""

pd.read_sql(q, im_db_conn)

In [ ]:
q = """
SELECT numvotes, primary_title, averagerating
FROM movie_ratings
    LEFT JOIN movie_basics
        USING(movie_id)
WHERE numvotes > 9999
;
"""
pd.read_sql(q, im_db_conn)
#movie_ratings_df.to_csv('data/imdb_movie_ratings')